# Test for the *differentiable* version of the Reach Operator from STREL

In [ ]:
import torch
import torch.nn as nn

## Differentiable Reach

In [ ]:
class DiffReach():
    """
    Reachability operator for STREL. Models bounded or unbounded reach
    over a spatial graph.
    """
    def __init__(
        self,
        adjacency_matrix,
        d1,
        d2,
        graph_nodes,
        is_unbounded: bool = False,
        distance_domain_min=None,
        distance_domain_max=None,
    ) -> None:
        super().__init__()
        self.d1 = d1
        self.d2 = d2
        self.is_unbounded = is_unbounded
        self.distance_domain_min = distance_domain_min
        self.distance_domain_max = distance_domain_max
        self.adjacency_matrix = adjacency_matrix
        self.graph_nodes = graph_nodes
        # inserisci min e max satisfaction domain
        # trova un modo per sostituire adj matrix in distance matrix
        # chiam al adj matrix weight matrix

    def __call__(self, s1, s2):
        return self._boolean(s1, s2)
        '''
        if quantitative:
            return self._quantitative(s1, s2)
        else:
            return self._boolean(s1, s2)
        '''

    def __str__(self) -> str:
        bound_type = "unbounded" if self.is_unbounded else f"[{self.d1},{self.d2}]"
        return f"Reach{bound_type}"

    def time_depth(self) -> int:
        return 0

    def neighbors_fn(self, node):
        aa = (self.adjacency_matrix[:, node] > 0)
        neigh = self.graph_nodes[aa]
        neigh_pairs = [(i.item(), self.adjacency_matrix[i, node].item()) for i in neigh]
        print('node = ', node, ' has neigh_pairs = ', neigh_pairs)
        return neigh_pairs

    def distance_function(self, weight):
        return weight

    # ----------------
    #    Boolean
    # ----------------

    def _boolean(self, s1, s2):
        if self.is_unbounded:
            return self._unbounded_reach_boolean(s1, s2)
        else:
            return self._bounded_reach_boolean(s1, s2)

    def _bounded_reach_boolean(self, s1, s2):
        # Initialize s with requires_grad=True
        s = torch.zeros(len(self.graph_nodes), requires_grad=True)

        # Set initial values without breaking the graph
        for i, lt in enumerate(self.graph_nodes):
            l = lt.item()
            if self.d1 == self.distance_domain_min:
                s = s.clone().scatter_(0, torch.tensor([l]), s2[l])
                #print('i: ', i, 'l: ', l, 's = ', s)
            else:
                s = s.clone().scatter_(0, torch.tensor([l]), self.distance_domain_min) # TODO: modifica distance_domain_min con boolean_min_satisfaction, crealo nelle proprietà. Per quantitative metti quantitative_min_satisfaction

        Q = {llt.item(): [(s2[llt.item()], self.distance_domain_min)] for llt in self.graph_nodes}

        while Q:
            print('Q = ', Q)
            Q_prime = {}
            for l in Q.keys():
                print('l = ', l)
                for v, d in Q[l]:
                    for l_prime, w in self.neighbors_fn(l):
                        print(f'neigh of {l}: {l_prime}')
                        v_new = torch.minimum(v, s1[l_prime])
                        d_new = d + w

                        if self.d1 <= d_new <= self.d2:
                            current_val = s[l_prime]
                            new_val = torch.maximum(current_val, v_new)
                            # Use scatter_ to preserve differentiability
                            s = s.clone().scatter_(0, torch.tensor([l_prime]), new_val)
                            print('s = ', s)

                        if d_new < self.d2:

                            # Create new list with updated values
                            existing_entries = Q_prime.get(l_prime, [])
                            updated = False
                            new_entries = []
                            for vv, dd in existing_entries:
                                if dd == d_new:
                                    new_v = torch.maximum(vv, v_new)
                                    new_entries.append((new_v, dd))
                                    updated = True
                                else:
                                    new_entries.append((vv, dd))

                            if not updated:
                                new_entries.append((v_new, d_new))
                            Q_prime[l_prime] = new_entries
            print('Q_prime = ', Q_prime)
            Q = Q_prime
        return s

    def _unbounded_reach_boolean(self, s1, s2):
        # Step 1: Inizializzazione di s
        # s = torch.zeros(len(self.graph_nodes), requires_grad=True)

        if self.d1 == self.distance_domain_min:
            s = s2 # s = s.clone().scatter_(0, torch.tensor([l]), s2)
        else:
            # Calcolo di d_max come massimo peso di un arco
            '''
            d_max = max(
                [
                    self.distance_function(self.adjacency_matrix[i, j].item())
                    for i in range(self.adjacency_matrix.shape[0])
                    for j in range(self.adjacency_matrix.shape[1])
                    if self.adjacency_matrix[i, j] > 0
                ]
            )
            '''
            d_max = torch.max(self.distance_function(self.weight_matrix)) # TODO: weight_matrix al momento non esiste, crealo
            # Chiamata al reach bounded tra [d1, d1 + d_max]
            self.d2 = self.d1 + d_max  # estendiamo temporaneamente l'intervallo
            s = self._bounded_reach(s1, s2)

        # Step 2: T = L (dizionario con chiavi = tutti i nodi)
        # Usiamo come valore semplicemente True, perché serve solo a garantire
        # l’unicità delle chiavi.
        T = {n.item(): True for n in self.graph_nodes}

        # Step 3: Iterazione fino a convergenza
        while T:
            # Inizializzo T_prime come dizionario vuoto
            T_prime = {}

            # Itero sulle chiavi di T
            for l in T.keys():
                for l_prime, w in self.neighbors_fn(l):
                    v_prime = torch.minimum(s[l], s1[l_prime])
                    v_prime = torch.maximum(v_prime, s[l_prime])

                    if v_prime != s[l_prime]: # forse meglio usare not torch.equal() ???
                        # Aggiorno il valore corrispondente in s
                        s = s.clone().scatter_(0, torch.tensor([l_prime]), v_prime)
                        # Inserisco l_prime in T_prime (la chiave rimane unica)
                        T_prime[l_prime] = True

            # Passo il dizionario T_prime a T per la prossima iterazione
            T = T_prime

        return s

    # ---------------------
    #     Quantitative
    # ---------------------

    def _quantitative(self, s1, s2):
        if self.is_unbounded:
            return self._unbounded_reach_quantitative(s1, s2)
        else:
            return self._bounded_reach_quantitative(s1, s2)

    def _bounded_reach_quantitative(self, s1, s2):
        # Inizializzo s con il minimo del dominio di distanza (robustezza minima)
        s = torch.full((len(self.graph_nodes),), self.distance_domain_min, requires_grad=True) # TODO: metti minimum satisfaction domain

        # Set iniziale: se d1 coincide col minimo dominio, copio s2; altrimenti mantengo distance_domain_min
        for i, lt in enumerate(self.graph_nodes):
            l = lt.item()
            if self.d1 == self.distance_domain_min:
                s = s.clone().scatter_(0, torch.tensor([l]), s2[l])
            # TODO: manca else

        # Coda Q: per ogni nodo, lista di coppie (robustezza, distanza)
        Q = {
            llt.item(): [(s2[llt.item()], self.distance_domain_min)]
            for llt in self.graph_nodes
        }

        while Q:
            Q_prime = {}
            for l in Q.keys():
                for (v, d) in Q[l]:
                    for l_prime, w in self.neighbors_fn(l):
                        v_new = torch.minimum(v, s1[l_prime])
                        d_new = d + w

                        # Se d_new è nell’intervallo [d1, d2], aggiorno s[l_prime] con max(current, v_new)
                        if self.d1 <= d_new <= self.d2:
                            current_val = s[l_prime]
                            new_val = torch.maximum(current_val, v_new)
                            s = s.clone().scatter_(0, torch.tensor([l_prime]), new_val)

                        # Se posso ancora estendere il percorso (d_new < d2), metto in Q_prime
                        if d_new < self.d2:
                            existing_entries = Q_prime.get(l_prime, [])
                            updated = False
                            new_entries = []
                            for (vv, dd) in existing_entries:
                                if dd == d_new:
                                    # Unisce le due robustezze scegliendo il massimo
                                    new_v = torch.maximum(vv, v_new)
                                    new_entries.append((new_v, dd))
                                    updated = True
                                else:
                                    new_entries.append((vv, dd))
                            if not updated:
                                new_entries.append((v_new, d_new))
                            Q_prime[l_prime] = new_entries
            Q = Q_prime

        return s

    def _unbounded_reach_quantitative(self, s1, s2):
        # Step 1: Inizializzazione di s
        # s = torch.full((len(self.graph_nodes),), self.distance_domain_min, requires_grad=True)

        if self.d1 == self.distance_domain_min:
            s = s2
        else:
            # Calcolo di d_max come massimo peso di un arco
            d_max = max(
                [
                    self.distance_function(self.adjacency_matrix[i, j].item())
                    for i in range(self.adjacency_matrix.shape[0])
                    for j in range(self.adjacency_matrix.shape[1])
                    if self.adjacency_matrix[i, j] > 0
                ]
            )
            # Estendo temporaneamente l’intervallo e richiamo bounded quantitativo
            self.d2 = self.d1 + d_max
            s = self._bounded_reach_quantitative(s1, s2)

        # Step 2: T = L (dizionario con chiavi = tutti i nodi)
        T = {n.item(): True for n in self.graph_nodes}

        # Step 3: Iterazione fino a convergenza
        while T:
            T_prime = {}
            for l in T.keys():
                for l_prime, w in self.neighbors_fn(l):
                    # v_prime = min(s[l], s1[l_prime])
                    v_prime = torch.minimum(s[l], s1[l_prime])
                    # poi confronto con s[l_prime] (robustezza massima fra i due)
                    new_val = torch.maximum(v_prime, s[l_prime])
                    if new_val != s[l_prime]:
                        s = s.clone().scatter_(0, torch.tensor([l_prime]), new_val)
                        T_prime[l_prime] = True
            T = T_prime

        return s

## Test

In [ ]:
# 1. Inizializzazione del grafo (16 nodi, 0-based indexing)
graph_nodes = torch.tensor(list(range(16)))
N = len(graph_nodes)

# Matrice di adiacenza
adjacency_matrix = torch.zeros((N, N))


edges = [
    (7, 0), (7, 3), (7, 5), (7, 6), (6, 7), (6, 4), (4, 6), (4, 5),
    (6, 1), (6, 9), (9, 6), (9, 2), (9, 13), (9, 8), (8, 9), (8, 13),
    (9, 15), (15, 9), (9,10), (10,9), (15,10), (10,15), (15,12),
    (10,14), (10,11),
    (0, 7), (3, 7), (5, 7), (5, 4),
    (1, 6), (2, 9), (13, 9), (13, 8),
    (12, 15), (14, 10), (11, 10)
]

for i, j in edges:
    adjacency_matrix[i, j] = 1

# 2. Ruoli:
#     - router (può inoltrare dati)
#     - coordinatore (inizializza la rete)
#     - end device (può solo ricevere, non inoltra)

router_nodes = [4, 6, 7, 8, 10, 15] # nodi 5, 7, 8, 9, 11, 16
coord_node = 9 # nodo 10
end_dev_nodes = [0, 1, 2, 3, 5, 11, 12, 13, 14] # nodi 1,2,3,4,6,12,13,14,15

# 3. Segnali: s1 tutti 1 (nessun vincolo sul sul tipo di nodo da cui parte il path), s2 solo router (nodo che voglio raggiungere)
# s1 = torch.ones(N)
s1 = torch.zeros(N)
s2 = torch.zeros(N)
for i in end_dev_nodes:
    s1[i] = 1
for i in router_nodes:
    s2[i] = 1
# s2 = torch.zeros(N)
# for i in router_nodes:
    # s2[i] = 1

# 4. Operatore NonDiffReach
reach_op = DiffReach(
    adjacency_matrix=adjacency_matrix,
    d1=0, d2=1, # al max 1 hop
    graph_nodes=graph_nodes,
    is_unbounded=False,
    distance_domain_min=0,
    distance_domain_max=1,
)

# 5. Calcolo della formula: s1 R[0,1] s2
result = reach_op(s1, s2) # restituisce un tensore di dimensione N (16): result[i] = 1 se il nodo i può raggiungere un router in 0 o 1 hop, result[i] = 0 altrimenti

# 6. Report: verifica solo per gli end devices
print("\nVerifica: ogni end device può raggiungere un router entro 1 hop?\n")
for i in end_dev_nodes:
    print(f"End device {i+1} → {'✅ Sì' if result[i]==1 else '❌ No'}")

# 7. Matrice Q per logging
Q = torch.zeros((3, N))
for i in range(N):
    Q[0, i] = i
    Q[1, i] = result[i].item()
    Q[2, i] = 1 if result[i] == 1 else 0

print("\nMatrice Q (nodo, soddisfazione, distanza):")
print(Q)

Q =  {0: [(tensor(0.), 0)], 1: [(tensor(0.), 0)], 2: [(tensor(0.), 0)], 3: [(tensor(0.), 0)], 4: [(tensor(1.), 0)], 5: [(tensor(0.), 0)], 6: [(tensor(1.), 0)], 7: [(tensor(1.), 0)], 8: [(tensor(1.), 0)], 9: [(tensor(0.), 0)], 10: [(tensor(1.), 0)], 11: [(tensor(0.), 0)], 12: [(tensor(0.), 0)], 13: [(tensor(0.), 0)], 14: [(tensor(0.), 0)], 15: [(tensor(1.), 0)]}
l =  0
node =  0  has neigh_pairs =  [(7, 1.0)]
neigh of 0: 7
s =  tensor([0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1.],
       grad_fn=<ScatterBackward0>)
l =  1
node =  1  has neigh_pairs =  [(6, 1.0)]
neigh of 1: 6
s =  tensor([0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1.],
       grad_fn=<ScatterBackward0>)
l =  2
node =  2  has neigh_pairs =  [(9, 1.0)]
neigh of 2: 9
s =  tensor([0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1.],
       grad_fn=<ScatterBackward0>)
l =  3
node =  3  has neigh_pairs =  [(7, 1.0)]
neigh of 3: 7
s =  tensor([0., 0., 0., 0., 1., 0., 1., 1., 1., 

In [ ]:
# 1. Inizializzazione del grafo (16 nodi, 0-based indexing)
graph_nodes = torch.tensor(list(range(16)))
N = len(graph_nodes)
adjacency_matrix = torch.zeros((N, N))

edges = [
    (7, 0), (7, 3), (7, 5), (7, 6), (6, 7), (6, 4), (4, 6), (4, 5),
    (6, 1), (6, 9), (9, 6), (9, 2), (9, 13), (9, 8), (8, 9), (8, 13),
    (9, 15), (15, 9), (9, 10), (10, 9), (15, 10), (10, 15), (15, 12),
    (10, 14), (10, 11),
    (0, 7), (3, 7), (5, 7), (5, 4),
    (1, 6), (2, 9), (13, 9), (13, 8),
    (12, 15), (14, 10), (11, 10)
]
for i, j in edges:
    adjacency_matrix[i, j] = 1

# 2. Ruoli dei nodi
router_nodes = [4, 6, 7, 8, 10, 15]
coord_node = 9
end_dev_nodes = [0, 1, 2, 3, 5, 11, 12, 13, 14]

# 3. Node features: tipo (0=coord, 1=router, 2=end_dev) + battery ∈ [0,1]
node_features = torch.zeros((N, 2))  # [:, 0] = tipo nodo, [:, 1] = battery

for i in router_nodes:
    node_features[i, 0] = 1  # router
node_features[coord_node, 0] = 0  # coordinator
for i in end_dev_nodes:
    node_features[i, 0] = 2  # end device

# Livelli di batteria random
node_features[:, 1] = torch.rand(N)

# 4. Costruzione segnali s1 (end_dev) e s2 (router)
s1 = (node_features[:, 0] == 2).float()
s2 = (node_features[:, 0] == 1).float()

# 5. Operatore DiffReach
reach_op = DiffReach(
    adjacency_matrix=adjacency_matrix,
    d1=0, d2=1,  # massimo 1 hop
    graph_nodes=graph_nodes,
    is_unbounded=False,
    distance_domain_min=0,
    distance_domain_max=1,
)

# 6. Esecuzione: verifica se un end device raggiunge un router entro 1 hop
result = reach_op(s1, s2)

# 7. Verifica testuale
print("\nVerifica: ogni end device può raggiungere un router entro 1 hop?\n")
for i in end_dev_nodes:
    reachable = result[i].item() > 0.5
    print(f"End device {i+1} → {'✅ Sì' if reachable else '❌ No'}")

# 8. Costruzione matrice Q (per logging/debug)
Q = torch.stack([
    torch.arange(N),              # ID nodo
    result,                       # soddisfazione della formula
    (result > 0.5).float(),       # binarizzazione
])
print("\nMatrice Q (nodo, soddisfazione, distanza):")
print(Q)

Q =  {0: [(tensor(0.), 0)], 1: [(tensor(0.), 0)], 2: [(tensor(0.), 0)], 3: [(tensor(0.), 0)], 4: [(tensor(1.), 0)], 5: [(tensor(0.), 0)], 6: [(tensor(1.), 0)], 7: [(tensor(1.), 0)], 8: [(tensor(1.), 0)], 9: [(tensor(0.), 0)], 10: [(tensor(1.), 0)], 11: [(tensor(0.), 0)], 12: [(tensor(0.), 0)], 13: [(tensor(0.), 0)], 14: [(tensor(0.), 0)], 15: [(tensor(1.), 0)]}
l =  0
node =  0  has neigh_pairs =  [(7, 1.0)]
neigh of 0: 7
s =  tensor([0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1.],
       grad_fn=<ScatterBackward0>)
l =  1
node =  1  has neigh_pairs =  [(6, 1.0)]
neigh of 1: 6
s =  tensor([0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1.],
       grad_fn=<ScatterBackward0>)
l =  2
node =  2  has neigh_pairs =  [(9, 1.0)]
neigh of 2: 9
s =  tensor([0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1.],
       grad_fn=<ScatterBackward0>)
l =  3
node =  3  has neigh_pairs =  [(7, 1.0)]
neigh of 3: 7
s =  tensor([0., 0., 0., 0., 1., 0., 1., 1., 1., 

In [ ]:
'''
    def _unbounded_reach(self, s1, s2):
        # Step 1: Inizializzazione di s
        s = torch.zeros(len(self.graph_nodes), requires_grad=True)

        if self.d1 == self.distance_domain_min:
            s = s2.clone()
        else:
            # Calcolo di d_max come massimo peso di un arco
            d_max = max(
                [
                    self.distance_function(self.adjacency_matrix[i, j].item())
                    for i in range(self.adjacency_matrix.shape[0])
                    for j in range(self.adjacency_matrix.shape[1])
                    if self.adjacency_matrix[i, j] > 0
                ]
            )
            # Chiamata al reach bounded tra [d1, d1 + d_max]
            self.d2 = self.d1 + d_max
            s = self._bounded_reach(s1, s2)

        # Step 2: T = L (insieme di tutti i nodi)
        T = set(n.item() for n in self.graph_nodes)

        # Step 3: Iterazione fino a convergenza
        while T:
            T_prime = set()
            for l in T:
                for l_prime, w in self.neighbors_fn(l):
                    v_prime = torch.minimum(s[l], s1[l_prime])
                    v_prime = torch.maximum(v_prime, s[l_prime])
                    if v_prime != s[l_prime]:
                        s = s.clone().scatter_(0, torch.tensor([l_prime]), v_prime)
                        T_prime.add(l_prime)
            T = T_prime

        return s
'''